In [1]:
cd ../..

c:\Users\Josu\Documents\Workspace\Human-Benchmark\src\server\model


In [ ]:
! del /q models\logs\*
! for /d %x in (models\logs\*) do @rd /get_model

In [ ]:
%load_ext teget_model
# %tensorboard --logdir=models/logs

In [5]:
import glob
from datetime import datetime

import chess
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from tensorflow.keras import callbacks, utils

from models import get_model
from scripts import absolute_bitboard

In [ ]:
XX, yy = [], []
for file in glob.glob("data/npz/*.npz"):
    with np.load(file) as f:
        XX.append(f["X"]), yy.append(f["y"])
X, y = np.concatenate(XX), np.concatenate(yy)

In [ ]:
_, x, _ = plt.hist(y, bins=75, density=True)  
ax = plt.plot(x,  stats.gaussian_kde(y)(x))

In [ ]:
model = get_modelget_model()
model.summary()

In [ ]:
utils.plot_model(model, to_file=f"models/png/model.png")

In [ ]:
tensor_board = callbacks.TensorBoard(
    log_dir=f"models/logs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
)
# model_checkpoint = callbacks.ModelCheckpoint(
#     f"models/ser/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
# )

In [ ]:
model.compile(
    optimizer="adam",
    loss="mse"
    # metrics=("mae",)
)

In [ ]:
_ = model.fit(
    x=X, y=y,
    batch_size=256,
    epochs=5,
    # callbacks=(tensor_board, model_checkpoint),# callbacks=(tensor_board, model_checkpoint),
    validation_split=.2
)

In [ ]:
model.save("models/ser/v3")

In [ ]:
# TEST
model = keras.Sequential(
    layers=(
    	Conv2D(32, (3, 3), activation="relu", input_shape=(8, 8, 6)),
    	Conv2D(64, (3, 3), activation="relu"),
    	GlobalAveragePooling2D(),
    	Dense(32, activation="relu"),
    	Dense(1, activation="tanh")
    )
)
model.summary()